In [1]:
import pandas as pd
import os
from copy import deepcopy
import numpy as np
from math import isclose

os.chdir("..")
from adaptive_multigrid import give_data_individual_names, find_best_individual_name, build_individual_dict_from_param_grid, update_param_grid
from gvs_threshold_calc import calc_gvs_thresh_ineq_met
from model.config.params import *


fitness_func = calc_gvs_thresh_ineq_met
sweep = "gateway_viability_sweep_ag3_"
kpis = pd.read_csv("simulation_data/{}.csv".format(sweep), index_col=0)
exec("param_config = {}".format(sweep))
next_name = sweep[:-2] + str(int(sweep[-2]) + 1) + "_"

variable_params = ['session_token_bucket_coefficient', 'gateway_fee_per_relay', 'application_fee_per_relay', 'gateway_minimum_stake', 'minimum_application_stake']
control_params = ['application_max_number', 'relays_per_session_gamma_distribution_scale']

params_to_evolve = {}
for x in variable_params:
    params_to_evolve[x] = param_config[x]

params_for_setup_only = {}
for x in control_params:
    params_for_setup_only[x] = param_config[x]

initial_param_grid = params_to_evolve
other_params_to_sweep = params_for_setup_only




/Users/seanmcowen/opt/anaconda3/envs/BlockScience/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
new_param_grid = deepcopy(initial_param_grid)
new_best_name = ""
new_best_fitness = -np.inf
times_repeat_best_name = 0
times_repeat_best_fitness = 0

old_param_grid = deepcopy(new_param_grid)
old_best_name = new_best_name
old_best_fitness = new_best_fitness

individual_dict = build_individual_dict_from_param_grid(old_param_grid)
        
        

data = kpis
data = give_data_individual_names(data)

new_best_name, new_best_val = find_best_individual_name(
            data, fitness_func=fitness_func, name_biased_towards=old_best_name
        )

assert not (
            new_best_name is None
        ), "The best individual should be somewhere in this dictionary."
print(new_best_name)

new_best_individual = individual_dict[new_best_name]

if new_best_name == old_best_name:
    times_repeat_best_name = times_repeat_best_name + 1
else:
    times_repeat_best_name = 0

if isclose(new_best_fitness, old_best_fitness):
    times_repeat_best_fitness = times_repeat_best_fitness + 1
else:
    times_repeat_best_fitness = 0
    
new_param_grid = update_param_grid(
                old_param_grid=old_param_grid, best_individual=new_best_individual
            )

Building the individual dict.
Individual dict is {'stbc:118.75_gfpr:20.0_afpr:25.0_gms:125000000000.0_ams:175000000000.0': {'session_token_bucket_coefficient': 118.75, 'gateway_fee_per_relay': 20.0, 'application_fee_per_relay': 25.0, 'gateway_minimum_stake': 125000000000.0, 'minimum_application_stake': 175000000000.0}, 'stbc:118.75_gfpr:20.0_afpr:25.0_gms:125000000000.0_ams:200000000000.0': {'session_token_bucket_coefficient': 118.75, 'gateway_fee_per_relay': 20.0, 'application_fee_per_relay': 25.0, 'gateway_minimum_stake': 125000000000.0, 'minimum_application_stake': 200000000000.0}, 'stbc:118.75_gfpr:20.0_afpr:25.0_gms:150000000000.0_ams:175000000000.0': {'session_token_bucket_coefficient': 118.75, 'gateway_fee_per_relay': 20.0, 'application_fee_per_relay': 25.0, 'gateway_minimum_stake': 150000000000.0, 'minimum_application_stake': 175000000000.0}, 'stbc:118.75_gfpr:20.0_afpr:25.0_gms:150000000000.0_ams:200000000000.0': {'session_token_bucket_coefficient': 118.75, 'gateway_fee_per_re

We have added individual names for the various parameter combinations.
The DataFrame now looks like                                                     Net Minting Rate  \
individual_name                                                        
stbc:118.75_gfpr:20.0_afpr:25.0_gms:12500000000...          0.000006   
stbc:118.75_gfpr:20.0_afpr:25.0_gms:12500000000...          0.000007   
stbc:118.75_gfpr:20.0_afpr:25.0_gms:12500000000...          0.000006   
stbc:118.75_gfpr:20.0_afpr:25.0_gms:12500000000...          0.000006   
stbc:118.75_gfpr:20.0_afpr:25.0_gms:12500000000...          0.000006   

                                                       KPI C  \
individual_name                                                
stbc:118.75_gfpr:20.0_afpr:25.0_gms:12500000000...  0.175676   
stbc:118.75_gfpr:20.0_afpr:25.0_gms:12500000000...  0.207684   
stbc:118.75_gfpr:20.0_afpr:25.0_gms:12500000000...  0.207438   
stbc:118.75_gfpr:20.0_afpr:25.0_gms:12500000000...  0.159647   
stbc:118.75

In [3]:
from psuu import build_next_param_config_code

build_next_param_config_code(
    next_name, new_param_grid, control_params, param_config
)

Add the following to model/config/params.py:

gateway_viability_sweep_ag4_ = build_params("Base")
gateway_viability_sweep_ag4_["session_token_bucket_coefficient"] = [165.625, 212.5]
gateway_viability_sweep_ag4_["gateway_fee_per_relay"] = [21.25, 22.5]
gateway_viability_sweep_ag4_["application_fee_per_relay"] = [25.0, 26.25]
gateway_viability_sweep_ag4_["gateway_minimum_stake"] = [137500000000.0, 150000000000.0]
gateway_viability_sweep_ag4_["minimum_application_stake"] = [187500000000.0, 200000000000.0]
gateway_viability_sweep_ag4_["application_max_number"] = [5, 20, 100]
gateway_viability_sweep_ag4_["relays_per_session_gamma_distribution_scale"] = [100000, 300000, 900000]
create_sweep("gateway_viability_sweep_ag4_",gateway_viability_sweep_ag4_,config_option_map_sweep)


Add the following to model/config/experiment.py:

for i in range(1, 289):
    experimental_setups["gateway_viability_sweep_ag4_{}".format(i)] = {
        "config_option_state": "Base",
        "config_option_params": "g